In [ ]:
from cracknuts.trace import ScarrTraceDataset
# 加载 scarr 格式的数据集
ds = ScarrTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313024458.zarr')
# ds = ScarrTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313022104.zarr')

In [ ]:
# Print dataset info
ds.info()

In [ ]:
# Show trace panel

from cracknuts.jupyter import display_trace_panel

tp = display_trace_panel(ds)
tp

In [ ]:
# Change trace index.
# show_trace 接收两个索引参数，第一个参数是通道索引，第二个是曲线索引，两个参数支持高级索引
# 需要注意的是通道A和B的名称在数据里为0和1，而其索引则根曲线文件中存储的通道数量有关系，如果只有一个通道时，无论哪个通道索引都为0，如果同时有A B两个通道则A通道索引为0，B通道索引为1，可以通过 ds.channel_names 进行确认当前曲线存储的时哪个通道

# 展示第一通道的前10条曲线
# tp.show_trace[1, :10]
# 展示第一通道的后10条曲线
# tp.show_trace[1, -10:]
# 展示第一通道的 10-20 曲线
# tp.show_trace[1, 10:20]
# 展示所有(两个)通道的前10条中的偶数位曲线
tp.show_trace[:, :10:2]

In [ ]:
# 获取第一通道的前10条曲线数据（np.ndarray）
ds.trace[1, :10].shape

In [ ]:
# 获取第一通道的前10条明文（当前版本该数据包含明文和密文）数据（np.ndarray）
ds.plain_text[1, :10].shape

In [ ]:
# 加载 numpy 格式 dataset
from cracknuts.trace import NumpyTraceDataset

ds2 = NumpyTraceDataset.load(r'D:\project\cracknuts\demo\jupyter\dataset\20250313152352.npy')

In [ ]:
# 获取第二通道的前10条曲线数据（np.ndarray）
ds2.trace[1, :10]

In [ ]:
# 获取第二通道的前10条明文（当前版本该数据包含明文和密文）数据（np.ndarray）
ds2.plain_text[1, :2]